In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from geotiff_processor import GeotiffProcessor
from ice_modeler import IceModeler

Using TensorFlow backend.


In [3]:
geotiff_index = pd.read_csv('daily_northern_geotiff_index.csv')
geotiff_index.date = pd.to_datetime(geotiff_index.date)
geotiff_index = geotiff_index.set_index('date')

In [4]:
geotiff_concentration = geotiff_index.loc[geotiff_index.image_type == 'concentration']

In [5]:
conc_index = geotiff_concentration.loc['1980-01-01':'2018-12-31',:]

In [6]:
conc_processor = GeotiffProcessor('raw_images/', image_index=conc_index)

In [8]:
conc_index = conc_processor.impute_missing_index_dates(set_this_index = True)
conc_index

,Unnamed: 0,file_name,hemisphere,image_type
date,,,,
1980-01-01,N_19800101_concentration_v3.0,daily/geotiff/1980/N_19800101_concentration_v3...,north,concentration
1980-01-02,N_19800103_concentration_v3.0,daily/geotiff/1980/N_19800103_concentration_v3...,north,concentration
1980-01-03,N_19800103_concentration_v3.0,daily/geotiff/1980/N_19800103_concentration_v3...,north,concentration
1980-01-04,N_19800103_concentration_v3.0,daily/geotiff/1980/N_19800103_concentration_v3...,north,concentration
1980-01-05,N_19800107_concentration_v3.0,daily/geotiff/1980/N_19800107_concentration_v3...,north,concentration
...,...,...,...,...
2018-12-27,N_20181227_concentration_v3.0,daily/geotiff/2018/N_20181227_concentration_v3...,north,concentration
2018-12-28,N_20181228_concentration_v3.0,daily/geotiff/2018/N_20181228_concentration_v3...,north,concentration
2018-12-29,N_20181229_concentration_v3.0,daily/geotiff/2018/N_20181229_concentration_v3...,north,concentration


In [10]:
conc_processor.load_default_colormaps(
    extent='default_extent_colormap.pkl',
    concentration='default_concentration_colormap.pkl')
# conc_last_ten = conc_processor.impute_missing_index_dates(set_this_index=True)

In [11]:
ice_modeler = IceModeler()

In [12]:
conc_images, years = conc_processor.process_images_channels_first_yearly_sample()

In [13]:
norm_images = ice_modeler.scale_to_normal(conc_images, image_type='concentration')

In [14]:
x_train = norm_images[:-6]
y_train = norm_images[1:-5]
x_test = norm_images[-6:-1]
y_test = norm_images[-5:]
pred_year = norm_images[-1]

In [17]:
x_train.shape

(33, 365, 1, 448, 304)

In [18]:
y_train.shape

(33, 365, 1, 448, 304)

In [20]:
ice_modeler.reset_model()
ice_modeler.add_n_td_conv2d(1, filters=4, kernel_size=(4,4), padding='same')
#ice_modeler.add_td_avg_pool(pool_size=(4,4), strides=2, padding='valid')
ice_modeler.add_td_flatten()
ice_modeler.add_n_tcn(1, nb_filters=4, kernel_size=7, nb_stacks=15)
ice_modeler.add_n_td_dense(1, ice_modeler.image_size)
ice_modeler.add_td_im_reshape()

In [21]:
ice_modeler.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
ice_modeler.fit(x_train, y_train, epochs=16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/16
33/33 [==============================] - 1450s 44s/step - loss: 0.0631
Epoch 2/16
33/33 [==============================] - 1499s 45s/step - loss: 0.0027
Epoch 3/16
33/33 [==============================] - 1531s 46s/step - loss: 0.0015
Epoch 4/16
33/33 [==============================] - 1547s 47s/step - loss: 0.0012
Epoch 5/16
33/33 [==============================] - 1517s 46s/step - loss: 0.0011
Epoch 6/16
33/33 [==============================] - 1498s 45s/step - loss: 0.0012
Epoch 7/16
33/33 [==============================] - 1463s 44s/step - loss: 0.0012
Epoch 8/16
33/33 [==============================] - 1472s 45s/step - loss: 0.0010
Epoch 9/16
33/33 [==============================] - 1465s 44s/step - loss: 0.0011
Epoch 10/16
33/33 [==============================] - 1459s 44s/step - loss: 0.0012
Epoch 11/16
33/33 [==============================] - 1462s 44s/step - loss: 0.0011
Epoch 12/16
33/33 [=============================

KeyboardInterrupt: 

In [ ]:
predictions = ice_modeler.predict(x_test)

In [ ]:
predictions.shape

In [ ]:
predictions_images = ice_modeler.scale_from_normal(predictions, image_type='concentration')

In [ ]:
for i in range(10):
    frame = predictions_images[1, i, :, :]
    actual = ice_modeler.scale_from_normal(y_test[0, i, :, :], image_type='concentration')
    conc_processor.make_colored_prediction_image(frame, image_type='concentration')
    conc_processor.make_colored_prediction_image(actual, image_type='concentration')
